In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

```bash
wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz
```

```bash
wc -l fhvhv_tripdata_2021-01.csv
```

In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [6]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [7]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

```bash
head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv
```

In [9]:
import pandas as pd

In [10]:
df_pandas = pd.read_csv('head.csv')

In [11]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [12]:
spark.createDataFrame(df_pandas).schema

C:\Users\kumar\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
C:\Users\kumar\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

Integer - 4 bytes
Long - 8 bytes

In [13]:
from pyspark.sql import types

In [14]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [15]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [16]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [17]:
df = df.repartition(24)

In [18]:
df.write.parquet('fhvhv/2021/01/')

### 5.3.2

In [19]:
df = spark.read.parquet('fhvhv/2021/01/')

In [20]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



```sql
SELECT * FROM df WHERE hvfhs_license_num =  HV0003
```

In [21]:
from pyspark.sql import functions as F

In [29]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 00:16:19|2021-01-01 00:33:12|         107|          97|
|2021-01-04 21:52:43|2021-01-04 22:02:08|         132|         216|
|2021-01-02 00:00:33|2021-01-02 00:21:06|         188|          97|
|2021-01-01 01:59:51|2021-01-01 02:17:01|         241|         142|
|2021-01-04 13:44:19|2021-01-04 13:55:57|         165|         133|
|2021-01-02 18:37:47|2021-01-02 19:02:27|         237|          82|
|2021-01-04 08:36:37|2021-01-04 08:40:24|          10|         216|
|2021-01-04 14:19:03|2021-01-04 14:23:30|         259|         259|
|2021-01-02 18:46:36|2021-01-02 19:01:04|          61|         177|
|2021-01-01 05:55:48|2021-01-01 06:09:19|         167|          47|
|2021-01-01 07:17:55|2021-01-01 07:22:33|         186|         170|
|2021-01-01 19:45:33|2021-01-01 19:55:49|       

In [30]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-02 20:11:50|2021-01-02 20:19:46|          76|         222|   null|
|           HV0003|              B02883|2021-01-01 00:16:19|2021-01-01 00:33:12|         107|          97|   null|
|           HV0003|              B02871|2021-01-04 21:52:43|2021-01-04 22:02:08|         132|         216|   null|
|           HV0003|              B02872|2021-01-02 00:00:33|2021-01-02 00:21:06|         188|          97|   null|
|           HV0003|              B02877|2021-01-01 01:59:51|2021-01-01 02:17:01|         241|         142|   null|
|           HV0003|              B02765|2021-01-04 13:44:19|2021-01-04 13:55:57|

In [23]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [24]:
crazy_stuff('B02884')

's/b44'

In [25]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [26]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-02|  2021-01-02|          76|         222|
|  a/b43| 2021-01-01|  2021-01-01|         107|          97|
|  a/b37| 2021-01-04|  2021-01-04|         132|         216|
|  e/b38| 2021-01-02|  2021-01-02|         188|          97|
|  s/b3d| 2021-01-01|  2021-01-01|         241|         142|
|  s/acd| 2021-01-04|  2021-01-04|         165|         133|
|  e/b38| 2021-01-02|  2021-01-02|         237|          82|
|  e/b32| 2021-01-04|  2021-01-04|          10|         216|
|  e/9ce| 2021-01-01|  2021-01-01|         137|         170|
|  s/b36| 2021-01-04|  2021-01-04|         259|         259|
|  e/b47| 2021-01-02|  2021-01-02|          61|         177|
|  e/9ce| 2021-01-01|  2021-01-01|          39|          76|
|  s/b44| 2021-01-01|  2021-01-01|         167|          47|
|  e/b38| 2021-01-01|  2

```bash
head -n 10 head.csv
```

### END